In [4]:
import base64
import htisec
user=base64.b64decode(b'YWNoZWNj').decode()
pwd=base64.b64decode(b'YjgyMDkxMQ==').decode()

web1=htisec.htisec_web()
web1.login(user,pwd)

hh=web1.get_orderlist()
#df1=web1.get_orderhis(hh)
#hold=web1.get_hold(hh)

Login OK,JSESSIONID: 423312FEC2A78A581C994F7661BA4BB1.wfapp01
Login Time: 2018-03-28 15:07:37
Get_url https://futures.htisec.com/wf/enquiryOrderList.do 200,Re:[]


In [10]:
import pandas  as pd
res=pd.DataFrame(index=['hold','close','total'])
hh=web1.get_orderlist()
tradelist=web1.get_tradelist(hh)
hold=web1.get_hold(hh)

point1=50
charges=67.08
c_point=round(charges/point1,1)

#hold-----------
hold_b=hold.loc[hold.trade_qty>0,'trade_qty'].sum()
hold_s=hold.loc[hold.trade_qty<0,'trade_qty'].sum()
hold_t=hold_b+hold_s
hold_sum=sum(hold['trade_qty']*hold['trade_price'])
hold_cost=round(hold_sum/hold_t if hold_t!=0 else 0,1)
hold_cost=-hold_cost if hold_t<0 else hold_cost
hold_PointT=round(hold_cost+c_point if hold_t<0 else hold_cost+c_point,1)
res.loc['hold','Bqty']=hold_b
res.loc['hold','Sqty']=hold_s
res.loc['hold','Hqty']=hold_t
res.loc['hold','Sum']=hold_sum
res.loc['hold','Cost']=hold_cost
res.loc['hold','Charges']=abs(hold_t)*charges
res.loc['hold','PointT']=hold_PointT

#close-----------
close_b=tradelist.loc[tradelist.trade_qty>0,'trade_qty'].sum()
close_s=tradelist.loc[tradelist.trade_qty<0,'trade_qty'].sum()
close_t=min(close_b,-close_s)
lots_sum=sum(tradelist['trade_qty']*tradelist['trade_price'])
c_pointT=-lots_sum-abs(hold_sum) if hold_t<0 else hold_sum-lots_sum
c_point_net=c_pointT-close_t*c_point
c_profit=c_pointT*point1
c_profit_net=c_profit-close_t*charges
res.loc['close','Bqty']=close_b
res.loc['close','Sqty']=close_s
res.loc['close','Hqty']=close_t
res.loc['close','Sum']=lots_sum
res.loc['close','Charges']=close_t*charges
res.loc['close','PointT']=c_pointT
res.loc['close','Point_NET']=c_point_net
res.loc['close','Point_AVG']=round(c_pointT/close_t-c_point,1) if close_t!=0 else 0
res.loc['close','Profit']=c_profit
res.loc['close','Profit_NET']=c_profit_net
res.loc['close','Profit_AVG']=round((c_profit-close_t*charges)/close_t,2)

#total
total_t=max(close_b,-close_s)
Hqty=close_b+close_s
total_PointT=round((lots_sum+total_t*c_point)/abs(Hqty) if Hqty!=0 else 0,1)
Point_NET=-total_PointT+hold_PointT if Hqty<0 else total_PointT-hold_PointT
res.loc['total','Bqty']=close_b
res.loc['total','Sqty']=close_s
res.loc['total','Hqty']=Hqty
res.loc['total','Sum']=lots_sum
res.loc['total','Charges']=total_t*charges
res.loc['total','Cost']=round(lots_sum/abs(Hqty)  if Hqty!=0 else 0,1)
res.loc['total','PointT']=total_PointT
res.loc['total','Point_NET']=Point_NET

rng=range(8,-7,-1)
open_lots=range(0,14,2)
cols=[ str(i)+'W' for i in rng]
trade=pd.DataFrame(columns=cols)
for ll in open_lots:
    for i in rng:   
        qty=0
        if ll==0 and Hqty!=0:
            qty=abs(Hqty)
        elif ll>0:
            qty=ll
        if qty>0:            
            winp=(i*10000/point1-c_point_net)/qty+c_point
            if ll==0:
                p_cost=round(hold_cost+winp,1)
                index='Hold'+str(Hqty)
                trade.loc[index,str(i)+'W']=p_cost
            else:
                winp=round(winp,1)
                index='Lots'+str(ll)
                trade.loc[index,str(i)+'W']=winp

res

Get_url https://futures.htisec.com/wf/enquiryOrderList.do 200,Re:[]
Get_url https://futures.htisec.com/wf/enquiryOrderDetail.do?acc_order_no=1475&has_TS_falg=Y 200,Re:[]
Get_url https://futures.htisec.com/wf/enquiryOrderDetail.do?acc_order_no=1476&has_TS_falg=Y 200,Re:[]
Get_url https://futures.htisec.com/wf/enquiryOrderDetail.do?acc_order_no=1479&has_TS_falg=Y 200,Re:[]
Get_url https://futures.htisec.com/wf/enquiryOrderDetail.do?acc_order_no=1480&has_TS_falg=Y 200,Re:[]
Get_url https://futures.htisec.com/wf/enquiryOrderDetail.do?acc_order_no=1481&has_TS_falg=Y 200,Re:[]
Get_url https://futures.htisec.com/wf/enquiryOrderDetail.do?acc_order_no=1482&has_TS_falg=Y 200,Re:[]
Get_url https://futures.htisec.com/wf/enquiryOrderDetail.do?acc_order_no=1483&has_TS_falg=Y 200,Re:[]
Get_url https://futures.htisec.com/wf/enquiryOrderDetail.do?acc_order_no=1484&has_TS_falg=Y 200,Re:[]
{'HSIH8': 0, 'HSIJ8': 6}


,Bqty,Sqty,Hqty,Sum,Cost,Charges,PointT,Point_NET,Point_AVG,Profit,Profit_NET,Profit_AVG
hold,6.0,0.0,6.0,181320.0,30220.0,402.48,30221.3,NaN,NaN,NaN,NaN,NaN
close,54.0,-48.0,48.0,183964.0,NaN,3219.84,-2644.0,-2706.4,-56.4,-132200.0,-135419.84,-2821.25
total,54.0,-48.0,6.0,183964.0,30660.7,3622.32,30672.4,451.1,NaN,NaN,NaN,NaN


In [5]:
hold

,refno,product,trade_price,trade_qty,order_time,filled_qty,price,trade_time,T+1,bqty,cond,initiator,r_qty,sqty,status,valid
8,1300,HSIH8,30894,-1,2018-03-27 09:44:34.0,-1,30894,2018-03-27 09:44:36,NaN,-1.0,不适用,01-0202975-00,0.0,0.0,已成交,即日
9,1301,HSIH8,30879,-1,2018-03-27 09:46:13.0,-1,30879,2018-03-27 09:46:13,NaN,-1.0,不适用,01-0202975-00,0.0,0.0,已成交,即日


In [8]:
tradelist

,refno,product,trade_price,trade_qty,trade_time,filled_qty,price,order_time,T+1,bqty,cond,initiator,r_qty,sqty,status,valid
0,9412,HSIH8,30448,1,2018-03-05 09:16:56,1,30454,2018-03-05 09:16:56.0,NaN,0.0,不适用,01-0202975-00,0.0,-1.0,已成交,即日
1,9413,HSIH8,30462,-1,2018-03-05 09:19:14,-1,30462,2018-03-05 09:19:10.0,NaN,-1.0,不适用,01-0202975-00,0.0,0.0,已成交,即日
2,9414,HSIH8,30449,-1,2018-03-05 09:31:49,-1,30446,2018-03-05 09:31:49.0,NaN,-1.0,不适用,01-0202975-00,0.0,0.0,已成交,即日
3,9415,HSIH8,30537,-1,2018-03-05 09:33:11,-1,30535,2018-03-05 09:33:11.0,NaN,-1.0,不适用,01-0202975-00,0.0,0.0,已成交,即日
4,9416,HSIH8,30558,-1,2018-03-05 09:35:11,-1,30555,2018-03-05 09:35:11.0,NaN,-1.0,不适用,01-0202975-00,0.0,0.0,已成交,即日
5,9417,HSIH8,30555,-1,2018-03-05 09:35:22,-1,30555,2018-03-05 09:35:21.0,NaN,-1.0,不适用,01-0202975-00,0.0,0.0,已成交,即日
6,9418,HSIH8,30645,-1,2018-03-05 09:38:26,-1,30645,2018-03-05 09:38:26.0,NaN,-1.0,不适用,01-0202975-00,0.0,0.0,已成交,即日
7,9419,HSIH8,30640,-1,2018-03-05 09:38:32,-1,30640,2018-03-05 09:38:29.0,NaN,-1.0,不适用,01-0202975-00,0.0,0.0,已成交,即日
8,9420,HSIH8,30636,-1,2018-03-05 09:38:38,-1,30636,2018-03-05 09:38:37.0,NaN,-1.0,不适用,01-0202975-00,0.0,0.0,已成交,即日
9,9421,HSIH8,30632,-1,2018-03-05 09:38:44,-1,30632,2018-03-05 09:38:44.0,NaN,-1.0,不适用,01-0202975-00,0.0,0.0,已成交,即日


In [9]:
hh=web1.get_orderlist()
hh.loc[hh.refno==8730,'status']='Mark'


Get_url https://futures.htisec.com/wf/enquiryOrderList.do 200,Re:[]


'b820911'